# Cleaning

In this jupyter notebook a cleaning process is done on the dataframe explored and partly cleaned in 'code1_EDA'. The objectives are:
- Column name unification.ç
- Change how some columns show their data.
- Change the data type of some columns.
- .

In [ ]:
# Let's import the libraries that will be needed for this EDA.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sidetable

In [ ]:
# To show all the columns of our dataframe.
pd.options.display.max_columns=None

In [ ]:
# Create the first dataframe containing the data from the csv obtained from Gbif.
df_0 = pd.read_csv('../data/gbif_mosquito_data.csv', sep='\t')

# Check the first three rows to see how this dataframe looks like.
df_0.head(3)

In [ ]:
# Divide the column species into 'genus' and 'species' and keep only the species name replacing the cells in this column.
df_1['species']=df_1['species'].str.split(' ',expand=True, n=1)[1]
df_1.head(3)

In [ ]:
# Check again null values looking at its percentage.
df_1.stb.missing()

In [ ]:
# Check for outliers in month using another method.
mean_month=df_1['month'].mean()
std_month=df_1['month'].std()
upper=mean_month + std_month
lower=mean_month - std_month
ucb=mean_month + std_month * 3
lcb=mean_month - std_month * 3

In [ ]:
# Start the graph.
month_graph=sns.histplot(x=df_1['month'],kde=True)
graph.axvline(x=mean_month, c='red',label='mean')

# Plot its standard deviation.
graph.axvline(x=upper,c='green',label='std')
graph.axvline(x=lower, c='green')

# Plot its confidence interval to 99.7%
graph.axvline(x=lcb, c='orange',label='99 lower')
graph.axvline(x=ucb, c='orange',label='99 upper')

plt.legend()